In [7]:
#sairam
import math
import numpy as np
import torch
import torch.nn as nn
from einops import rearrange
from imageio import imwrite
from pydantic import validator
import kornia
import cv2
import math
import pdb
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from torchvision import transforms

from my.utils import (
    tqdm, EventStorage, HeartBeat, EarlyLoopBreak,
    get_event_storage, get_heartbeat, read_stats
)
from my.config import BaseConf, dispatch, optional_load_config
from my.utils.seed import seed_everything

from adapt import ScoreAdapter, karras_t_schedule
from run_img_sampling import SD, StableDiffusion
from misc import torch_samps_to_imgs
from pose import PoseConfig, camera_pose, sample_near_eye

from run_nerf import VoxConfig
from voxnerf.utils import every
from voxnerf.render import (
    as_torch_tsrs, rays_from_img, ray_box_intersect, render_ray_bundle
)
from voxnerf.vis import stitch_vis, bad_vis as nerf_vis, vis_img
from voxnerf.data import load_blender
from my3d import get_T, depth_smooth_loss
from open3d import *    

# # diet nerf
# import sys
# sys.path.append('./DietNeRF/dietnerf')
# from DietNeRF.dietnerf.run_nerf import get_embed_fn
# import torch.nn.functional as F

device_glb = torch.device("cuda")

In [2]:
from run_sjc_objaverse import SJC, render_one_view
from voxnerf.vis import export_movie
import matplotlib.pyplot as plt

In [3]:
cfg = optional_load_config(fname="full_config.yml")

loaded config
 
family: sd
sd:
  variant: objaverse
  v2_highres: false
  prompt: a photograph of an astronaut riding a horse
  im_path: data/nerf_synthetic/chair/train/r_2.png
  scale: 100.0
  precision: autocast
lr: 0.005
n_steps: 10000
vox:
  model_type: V_SD
  bbox_len: 1.0
  grid_size:
  - 100
  - 100
  - 100
  step_ratio: 0.5
  density_shift: -1.0
  ray_march_weight_thres: 0.0001
  c: 4
  blend_bg_texture: false
  bg_texture_hw: 4
  time_lr: 0.0005
pose:
  rend_hw: 32
  FoV: 49.1
  R: 2.0
  up: z
emptiness_scale: 10
emptiness_weight: 0
emptiness_step: 0.5
emptiness_multiplier: 20.0
grad_accum: 1
depth_smooth_weight: 10000.0
near_view_weight: 10000.0
depth_weight: 0
var_red: true
train_view: true
scene: elevendbeanie
index: 2
view_weight: 10000
prefix: experiments/exp_wild
nerf_path: data/nerf_wild



In [4]:
mod = SJC(**cfg)

In [5]:
family = cfg.pop("family")
model: ScoreAdapter = getattr(mod, family).make()
poser = mod.pose.make()


Loading model from ../stable-diffusion/last.ckpt
Global Step: 88460
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.53 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [75]:
def make_video(scene, cfname, path, time):
    vox = mod.vox.make()
    #load model
    state = torch.load(cfname, map_location="cpu")
    vox.load_state_dict(state)
    vox.to(device_glb)
    
    #PREPARE INPUT
    images_, _, poses_, times_, mask_, fov_x = load_blender('train', scene=scene, path=path)
    input_im_transformed = []
    with torch.no_grad():
        tforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop((256, 256))
        ])
        for img in images_:
            img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
            img = torch.as_tensor(img, dtype=float, device=device_glb)
            img = img.permute(2, 0, 1)[None, :, :]
            img = img * 2. - 1.
            input_im_transformed.append(tforms(img))

    #PLOT IMAGES
    plot_images = []
    with torch.no_grad():
        vox.eval()
        K, poses = poser.sample_test(100)
#         times = np.linspace(0.52,0.52,50)
        aabb = vox.aabb.T.cpu().numpy()
#         plot_images.append(input_im_transformed[condition_image_index].cpu().numpy()[0].transpose(1,2,0))
#         for idx in tqdm(range(len(times))):
#             if pose==26:
#                 import plotly
#                 from plotly import figure_factory
#                 import plotly.express as px

#                 idx = np.random.randint(og_pts.shape[0], size=300)
#                 dx, dy, dz = og_pds[idx,0], og_pds[idx,1], og_pds[idx,2]
#                 x, y, z = og_pts[idx,0], og_pts[idx,1], og_pts[idx,2]
#                 xdx, ydy, zdz = og_tps[idx,0], og_tps[idx,1], og_tps[idx,2]
#                 fig = plotly.graph_objects.Figure(data=[plotly.graph_objects.Scatter3d(x=x, y=y, z=z, mode='markers', marker_color=['red' for zzz in dz], name="x", text=idx, hoverinfo='text'),
#                                                         plotly.graph_objects.Scatter3d(x=dx, y=dy, z=dz, mode='markers', marker_color=["lightskyblue" for zzz in dz], name="dx"),
#                                                         plotly.graph_objects.Scatter3d(x=xdx, y=ydy, z=zdz, mode='markers', marker_color=["midnightblue" for zzz in dz], name="x+dx", text=idx, hoverinfo='text')])
#                 fig.write_html("fig.html")
#                 fig = figure_factory.create_quiver(x, y, dx, dy)
#                 fig.write_html("xy.html")
#                 fig = figure_factory.create_quiver(x, z, dx, dz)
#                 fig.write_html("xz.html")
#                 print(depth.clone().detach().cpu().numpy().flatten())
#                 plt.imsave("deptj.png",depth.clone().detach().cpu().numpy())
#                 plt.imsave("disp.png",disp.clone().detach().cpu().numpy())

#             pose = poses[pose]
        y, depth, disp = render_one_view(vox, aabb, poser.H, poser.W, K, poses[26], time)
        y = model.decode(y)
        plot_images.append((y.cpu().numpy()[0].transpose(1,2,0)).astype(np.float32))
    ffname = f"{scene}_time_change".replace(".","_")
#     pdb.set_trace()
    vox.export_mesh(f"{scene}_{time}.ply", threshold_mult=1.5)
    return plot_images, ffname, vox

In [86]:
vox1.color


Parameter containing:
tensor([[[[[-1.0750e+00, -1.2514e-01,  2.4148e+00,  ..., -1.5211e-02,
             4.8821e-01, -9.7113e-01],
           [-2.7817e+00, -1.4192e+00, -7.6331e-01,  ...,  5.4852e-01,
             9.8424e-01,  4.5790e-01],
           [-1.5393e+00,  1.0884e+00,  7.2990e-01,  ...,  4.6696e-01,
             8.1936e-01, -5.4975e-01],
           ...,
           [-3.0222e-02,  1.1255e-01, -4.1443e-01,  ...,  1.7039e+00,
             6.2235e-01,  2.0111e+00],
           [ 2.6562e-01,  2.3622e-01, -2.6868e-01,  ..., -3.5612e-01,
             2.3253e+00,  1.6243e+00],
           [-1.6219e+00,  2.3607e+00,  6.0885e-02,  ...,  5.7550e-01,
             1.8651e-01,  6.8958e-01]],

          [[-1.8286e+00, -1.7120e-01, -5.4024e-01,  ..., -7.2850e-02,
             1.7823e-01, -1.5070e-01],
           [ 5.7448e-01, -2.0524e-01, -5.5654e-01,  ...,  2.4320e+00,
             7.9870e-01, -1.0608e+00],
           [ 9.2681e-01, -2.0592e+00, -1.8137e+00,  ...,  9.2187e-01,
             2.299

In [87]:
torch.where((vox2.color == vox1.color)==False)

(tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64))

In [77]:
ckpt_fname = '/proj/vondrick3/sruthi/zero123-internal-test/sjc/experiments/exp_wild/tendpillowsquish_scale-100.0_train-view-True_view-weight-10000_depth-smooth-wt-10000.0_near-view-wt-10000.0_time-lr_0.001/ckpt/step_9000.pt'
plots, ffname, vox1=make_video('tendpillowsquish', ckpt_fname, 'data/nerf_wild', 0)
export_movie(plots, ffname.replace(".","_"))

[0 0 1]
> /tmp/ipykernel_1464391/1836922167.py(59)make_video()
     56         plot_images.append((y.cpu().numpy()[0].transpose(1,2,0)).astype(np.float32))
     57     ffname = f"{scene}_time_change".replace(".","_")
     58     pdb.set_trace()
---> 59     vox.export_mesh(f"{scene}_{time}.ply", threshold_mult=1.5)
     60     return plot_images, ffname, vox

ipdb> vox.desnity
*** AttributeError: 'V_SD' object has no attribute 'desnity'
ipdb> c
> /proj/vondrick3/sruthi/zero123-internal-test/sjc/voxnerf/vox.py(74)export_mesh()
     72     def export_mesh(self, path, reso_mult=2, threshold=None, threshold_mult=8, kernel_size=7, kernel_type="avg", erosion_kernel_size=5):
     73         pdb.set_trace()
---> 74         if reso_mult is None:
     75             sigma = self.density
     76             reso_mult = 1

ipdb> c
using density threshold: 0.3950408399105072


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [78]:
ckpt_fname = '/proj/vondrick3/sruthi/zero123-internal-test/sjc/experiments/exp_wild/tendpillowsquish_scale-100.0_train-view-True_view-weight-10000_depth-smooth-wt-10000.0_near-view-wt-10000.0_time-lr_0.001/ckpt/step_9000.pt'
plots, ffname, vox2=make_video('tendpillowsquish', ckpt_fname, 'data/nerf_wild', 0.52)
export_movie(plots, ffname.replace(".","_"))

[0 0 1]
> /tmp/ipykernel_1464391/1836922167.py(59)make_video()
     56         plot_images.append((y.cpu().numpy()[0].transpose(1,2,0)).astype(np.float32))
     57     ffname = f"{scene}_time_change".replace(".","_")
     58     pdb.set_trace()
---> 59     vox.export_mesh(f"{scene}_{time}.ply", threshold_mult=1.5)
     60     return plot_images, ffname, vox

ipdb> c
> /proj/vondrick3/sruthi/zero123-internal-test/sjc/voxnerf/vox.py(74)export_mesh()
     72     def export_mesh(self, path, reso_mult=2, threshold=None, threshold_mult=8, kernel_size=7, kernel_type="avg", erosion_kernel_size=5):
     73         pdb.set_trace()
---> 74         if reso_mult is None:
     75             sigma = self.density
     76             reso_mult = 1

ipdb> c
using density threshold: 0.3950408399105072


Lossy conversion from float32 to uint8. Range [-0.0003662109375, 1.03515625]. Convert image to uint8 prior to saving to suppress this warning.


In [70]:
vox

V_SD(
  (_time): ModuleList(
    (0): Linear(in_features=84, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): Linear(in_features=319, out_features=256, bias=True)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): Linear(in_features=256, out_features=256, bias=True)
  )
  (_time_out): Linear(in_features=256, out_features=3, bias=True)
)

In [ ]:

fig, axes = plt.subplots(5,5, sharex=True, sharey=True) 

for img, ax in zip(plots[75:100], axes.flat):
    ax.imshow(img)
    ax.axis('off')

> /proj/vondrick3/sruthi/zero123-internal-test/sjc/voxnerf/vox.py(74)export_mesh()
     72     def export_mesh(self, path, reso_mult=2, threshold=None, threshold_mult=8, kernel_size=7, kernel_type="avg", erosion_kernel_size=5):
     73         pdb.set_trace()
---> 74         if reso_mult is None:
     75             sigma = self.density
     76             reso_mult = 1

ipdb> c
using density threshold: 0.3950408399105072


In [66]:
cloud = open3d.io.read_point_cloud("temp0_52.ply")

In [68]:
cloud

PointCloud with 128696 points.

In [69]:
open3d.visualization.draw_geometries([cloud])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
